In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    return_messages=True,
    max_token_limit=120,
    memory_key='chat_history',
)

prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful AI talking to a human'),
    MessagesPlaceholder(variable_name="chat_history"),
    ('human', '{question}'),
])

def load_memory(input):
    return memory.load_memory_variables({})['chat_history']

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {'output': result.content})
    print(result)


invoke_chain("My name is James")


content='Hello James! How can I assist you today?'


In [2]:
invoke_chain("What is my name?")

content='Your name is James.'
